# TensorFlow 2 quickstart for experts

In [3]:
# Import 

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [4]:
# Import MNIST dataset.

mnist = tf.keras.datasets.mnist

# Split it into train and test split. 

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
# Add a channels dimension

x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [9]:
# We want to shuffle the data and create batches at the same time.  

# Create a shuffled dataset. 

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)

# Now create a test dataset.

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [12]:
# We will use classes to create the model. 

# Define the class here.

class NN_Model(Model):
    
    def __init__(self):
        
        super(NN_Model, self).__init__()
        self.conv1 = Conv2D(32,3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation = 'relu')
        self.d2 = Dense(10)
        
    def call(self, x):
        
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model

model = NN_Model()

In [13]:
# Here we set what we want the optimizer and Loss to be. 

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [14]:
# We create some metrics that will be used for reporting the loss and acc.

train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [16]:
# Here we set up the train step.

@tf.function
def train_step(images, labels):
    
  with tf.GradientTape() as tape:
    
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [18]:
# Here we set up the test step. 

@tf.function

def test_step(images, labels):
    
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [19]:
# Run the Model.

# Set the number of Epochs you want.

EPOCHS = 5



for epoch in range(EPOCHS):
    
  # Reset the metrics at the start of the next epoch

  train_loss.reset_states()
  train_accuracy.reset_states()

  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.1309851109981537, Accuracy: 96.07499694824219, Test Loss: 0.07083933055400848, Test Accuracy: 97.62999725341797
Epoch 2, Loss: 0.0390058197081089, Accuracy: 98.85333251953125, Test Loss: 0.05357753857970238, Test Accuracy: 98.33999633789062
Epoch 3, Loss: 0.02128186821937561, Accuracy: 99.2933349609375, Test Loss: 0.05841809883713722, Test Accuracy: 98.22999572753906
Epoch 4, Loss: 0.01260331179946661, Accuracy: 99.58333587646484, Test Loss: 0.05995478108525276, Test Accuracy: 98.25999450683594
Epoch 5, Loss: 0.008150392211973667, Accuracy: 99.74832916259766, Test Loss: 0.06747067719697952, Test Accuracy: 98.32999420166016


# This Completes TensorFlow 2 quickstart for experts